In [1]:
import warnings
import logging
warnings.filterwarnings("ignore", category=FutureWarning)

logging.disable(logging.NOTSET)


In [2]:
from pathlib import Path

import anndata as ad
import spatialtis as st
from sqlalchemy import create_engine
from dotenv import dotenv_values
from sscmap import Global, Record, Tech, Molecule, Species

config = dotenv_values(".env")
Global.engine = create_engine(f"postgres://{config['USER']}:{config['PASSWORD']}"
                              f"@{config['HOST']}/{config['DB_NAME']}")

Global.export = Path("/Volumes/MILK-SSD/aquila-static")
save_h5ad_path = Path("/Volumes/MILK-SSD/aquila-h5ad-data")

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:get_version:dirname: Trying to get version of get_version from dirname /Users/milk/anaconda3/envs/scpdb/lib/python3.8/site-packages
INFO:get_version:dirname: Failed; Does not match re.compile('get[_-]version-([\\d.]+?)(?:\\.dev(\\d+))?(?:[_+-]([0-9a-zA-Z.]+))?$')
INFO:get_version:git: Trying to get version from git in directory /Users/milk/anaconda3/envs/scpdb/lib/python3.8/site-packages
INFO:get_version:git: Failed; directory is not managed by git
INFO:get_version:metadata: Trying to get version for get_version in dir /Users/milk/anaconda3/envs/scpdb/lib/python3.8/site-packages
INFO:get_version:metadata: Succeeded
INFO:get_version:dirname: Trying to get version of legacy_api_wrap from dirname /Users/milk/anaconda3/envs/scpdb/lib/python3.8/site-packages
INFO:get_version:dirname: Failed; Does not match re.compile('le

In [20]:
data = ad.read_h5ad(save_h5ad_path / 'mibi_TNBC_data.h5ad')
r = Record(data, groups_keys=['Patient', 'Stage']).set_meta(
            doi="10.1016/j.cell.2018.08.039",
            technology=Tech.MIBI,
            species=Species.Human,
            tissue="breast",
            molecule=Molecule.Protein,
            disease="cancer",
            disease_subtype="breast cancer",
            resolution=260,
).ok(expand=30)

Data record c1afb1c043e9aa6e4c1624e040754788 has 184060 cells and 38 markers.


Find neighbors ██████████ 100% 00:00|00:01


co-expression ██████████ 100% 00:00|00:00


Calculating heterogeneity ██████████ 100% 00:00|00:14


Neighborhood analysis ██████████ 100% 00:00|00:06


In [21]:
data = ad.read_h5ad(save_h5ad_path / 'codex_spleen.h5ad')

In [22]:
data

AnnData object with n_obs × n_vars = 83786 × 30
    obs: 'ROI', 'centroid', 'cell_type'
    var: 'markers'

In [24]:
st.CONFIG.EXP_OBS = ['ROI']
st.find_neighbors(data, expand=5)

Find neighbors ██████████ 100% 00:00|00:00


In [25]:
data.obs.cell_neighbors_count.mean()

5.992815028763755

In [27]:
data.obs

,ROI,centroid,cell_type,neighbors_ix,cell_neighbors,cell_neighbors_count
cellID,,,,,,
1,BALBc-3,"(10, 70)",CD11c(+) B cells,0,"[0, 38410, 56371, 53419, 64307, 67291, 8286]",7
2,BALBc-3,"(1000, 294)",CD11c(+) B cells,1,"[69014, 1, 262, 63986, 36907, 263, 264]",7
3,BALBc-3,"(1003, 605)",CD11c(+) B cells,2,"[226, 2, 55720]",3
4,BALBc-3,"(1015, 873)",CD11c(+) B cells,3,"[650, 3, 37109, 670]",4
5,BALBc-3,"(1021, 953)",CD11c(+) B cells,4,"[626, 35396, 55819, 653, 73301, 4, 790]",7
...,...,...,...,...,...,...
83783,BALBc-3,"(914, 388)",CD4(+)MHCII(+),83781,"[30402, 35319, 83781]",3
83784,BALBc-3,"(961, 194)",CD4(+)MHCII(+),83782,"[31646, 83782, 54376, 31693, 79222, 31527, 637...",9
83785,BALBc-3,"(99, 18)",CD4(+)MHCII(+),83783,"[79404, 54680, 32088, 36871, 54353, 83783, 964...",8


In [ ]:
r = Record(data, groups_keys=['ROI']
          ).set_meta(doi="10.1016/j.cell.2018.07.010",
                    technology=Tech.CODEX,
                     species=Species.Mouse,
                    tissue="spleen",
                     molecule=Molecule.Protein,
                     disease="autoimmune",
                     disease_subtype="lupus",
                     resolution=188,
                    ).ok(expand=5)